In [ ]:
# Configuração do ambiente virtual
# python -m venv venv
# venv\Scripts\activate
# Instalação das bibliotecas necessárias
# %pip install pandas openpyxl numpy ipykernel nbformat plotly
# Verificar o interpretador Python
# import sys
# print(sys.executable)
import pandas as pd
#Read Excel
tabela = pd.read_excel("Abastecimento_Caminhao.xlsx")
tabelaPlaca = pd.read_excel("RELAÇÃO FROTA ATUALIZADO 251124.xlsx")
#Convrt'colun'Data Req.'p/type-datetime
tabela['Data Req.'] = pd.to_datetime(tabela['Data Req.'], errors='coerce', dayfirst=True)
#Excld coluns-unwanted
columns_exclud = ["Combustível", "Vlr. Unitário", "Hora Abast.", "Obs.", "Abast. Externo"]
current_columns = [col for col in columns_exclud if col in tabela.columns]
if current_columns:
    tabela = tabela.drop(columns=current_columns)
else:
    print("Nenhuma das colunas especificadas foi encontrada no DataFrame.")
#Ordnr dados pel'Data Req.'d-form-ascendnt
tabela = tabela.sort_values(by=['Data Req.'])
#Define-date p/filtr inic-final
dtini = "01/06/2024"  # Data inicial para cálculo
dtfinal = "01/07/2024"  # Data inicial para exibição
data_inicial_calculo = pd.to_datetime(dtini, format='%d/%m/%Y')
data_inicial_exibicao = pd.to_datetime(dtfinal, format='%d/%m/%Y')
data_final = pd.Timestamp.now()
#Aplic filtr d date p/cálc
tabela_calculo = tabela[(tabela['Data Req.'] >= data_inicial_calculo) & (tabela['Data Req.'] <= data_final)].copy()
#Calc dif d Km p/ each Veículo/Equip.
tabela_calculo['Diferença de Km'] = tabela_calculo.groupby('Veículo/Equip.')['Km Atual'].diff().abs()
#Get value Litros-abastcimnt before p/ each Veículo/Equip.
tabela_calculo['Litros Anterior'] = tabela_calculo.groupby('Veículo/Equip.')['Litros'].shift(1)
#Calc Km p-Litro
tabela_calculo['Km por Litro'] = tabela_calculo['Diferença de Km'] / tabela_calculo['Litros Anterior']
#Arredond values de Km por Litro
tabela_calculo['Km por Litro'] = tabela_calculo['Km por Litro'].round(3)
#Filtr dados p/exibiç startng-01/07/2024
tabela_exibicao = tabela_calculo[tabela_calculo['Data Req.'] >= data_inicial_exibicao]
#Reformat colun'Data Req.'p/exib formt desejd using .loc[]
tabela_exibicao.loc[:, 'Data Req.'] = tabela_exibicao['Data Req.'].dt.strftime('%d/%m/%Y')
#Mescl table'tabela_exibicao'c/table'tabelaPlaca'p/includ colun"PLACA/"
tabela_exibicao = tabela_exibicao.merge(tabelaPlaca[['Placa TOPCON', 'PLACA/']], left_on='Veículo/Equip.', right_on='Placa TOPCON', how='left')
#Reorganiz ordem-colun
colunas_ordem = ["Requisição", "Data Req.", "Requisitante", "PLACA/", "Diferença de Km", "Km por Litro", "Vlr. Total", "Km Atual", "Horim. Equip."]
colunas_restantes = [col for col in tabela_exibicao.columns if col not in colunas_ordem]
tabela_exibicao = tabela_exibicao[colunas_ordem + colunas_restantes]
#Exib table c/new-coluns
display(tabela_exibicao)

In [ ]:
import pandas as pd
#RD Excel
tabela = pd.read_excel("Abastecimento_Caminhao.xlsx")
tabelaPlaca = pd.read_excel("RELAÇÃO FROTA ATUALIZADO 251124.xlsx")
#Convert'Data Req.'p/datetime
tabela['Data Req.'] = pd.to_datetime(tabela['Data Req.'], errors='coerce', dayfirst=True)
#Exclud coluns unwantd
columns_exclud = ["Combustível", "Vlr. Unitário", "Hora Abast.", "Abast. Externo"]
current_columns = [col for col in columns_exclud if col in tabela.columns]
if current_columns:
    tabela = tabela.drop(columns=current_columns)
#Orden dados BY'Data Req.'forma ascend
tabela = tabela.sort_values(by=['Data Req.'])
tabela.head()
#Calc dif d Km /obt value d Litros do abastecimnt anterior p/cada Veículo/Equip.
tabela['Diferença de Km'] = tabela.groupby('Veículo/Equip.')['Km Atual'].diff().abs()
tabela['Litros Anterior'] = tabela.groupby('Veículo/Equip.')['Litros'].shift(1)
#Calc Km por Litro
tabela['Km por Litro'] = tabela['Diferença de Km'] / tabela['Litros Anterior']
tabela['Km por Litro'] = tabela['Km por Litro'].round(3)
#Calc Combustív Restante
tabela['Combustível Restante'] = tabela['Diferença de Km'] % tabela['Litros Anterior']
tabela['Combustível Restante'] = tabela['Combustível Restante'].round(3)
veiculoSearch = input("Veículo Desejado: ")
def listVeiculo(veiculoSearch, tabela):
    veiculoSelected = veiculoSearch
    if tabela['Veículo/Equip.'].str.contains(veiculoSearch, case=False, na=False).any():
        filtered_table = tabela[tabela['Veículo/Equip.'].str.contains(veiculoSearch, case=False, na=False)]
        display(filtered_table)
    else:
        print("Não há Veículos/Equipamentos com esse nome!")
listVeiculo(veiculoSearch, tabela)

In [ ]:
veiculoSearch = input("Veículo Desejado: ")
def listVeiculo(veiculoSearch, tabela):
    veiculoSelected = veiculoSearch
    if tabela['Veículo/Equip.'].str.contains(veiculoSearch, case=False, na=False).any():
        filtered_table = tabela[tabela['Veículo/Equip.'].str.contains(veiculoSearch, case=False, na=False)]
        display(filtered_table)
    else:
        print("Não há Veículos/Equipamentos com esse nome!")
listVeiculo(veiculoSearch, tabela)